# 천하제일 프롬프트 대회

- 이 게임은 2명이서 하는 1 vs 1 배틀 게임입니다.

- 우선 두 분이서 각각 원하는 프롬프트를 통해서 각자의 캐릭터를 만드실 수 있습니다.
    - ex) 세상에서 가장 귀여운 고양이 키티, 이 고양이는 애교로 세상을 멸망시킬 존재이다.

    - ex) 악어와 기관총을 합친 악어들의 왕, 봄바르딜로 크로코딜로 그의 기관총은 모든 적을 무자비하게 죽여버린다.

- 그러면 openai를 통해 작성하신 프롬프트에 걸맞는 본인 캐릭터의 능력치와, 필살 스킬이 자동으로 정해집니다.
    ex) 
    
        player1

        캐릭터 이름: 키티키티
        체력: 140
        공격력: 18
        방어력: 12
        회피율: 30
        특수공격 스킬: 애교 대멸종: 상대의 공격력을 현재 수치의 50%만큼 즉시 감소시키고, 자신의 회피율을 25 증가시킨 후, 감소시킨 공격력 수치의 100%만큼 추가 피해를 입힌다.

        player2

        캐릭터 이름: 봄바르딜로 크로코딜로
        체력: 230
        공격력: 40
        방어력: 28
        회피율: 12
        특수공격 스킬: 탄막제왕: 기관총을 과열할 만큼 난사한다. 자신의 공격력을 35 증가시킨 후, 상대에게 증가된 공격력의 120%만큼 피해를 입힌다. 반동으로 2턴 동안 자신의 회피율이 10 감소하고 방어력이 5 감소한다.

- 전투는 기본적으로 시뮬레이션을 통해 진행되며, 상대방에게 70 이상의 데미지를 넣거나, 상대방의 스킬을 받아치는 스킬을 가지고 있는 경우 본인의 필살 스킬이 자동으로 발동됩니다.

- 정해진 능력치와 필살 스킬을 통해 상대방을 이기고, 프롬프트의 왕이 되세요!

In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

# 캐릭터 생성

In [2]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

template = """다음은 사용자가 입력으로 ###{input}###이라고 input을 얘기할 때, 이를 바탕으로 캐릭터를 생성하고, 캐릭터에 맞는 능력치를 부여하는 예시입니다.
능력 부여는 입력으로 들어온 프롬프트에 따라 당신이 마음대로 부여하는 것입니다.
 능력치는 각각 hp, 공격력, 방어력,회피율과 특수공격을 위한 스킬이 주어집니다.
 특수공격을 제외한 능력치는 다음과 같이 부여하세요. HP는 기본적으로 100~300이어야 하며 공격력은 20~40, 방어력은 10~30, 회피율은 5~18 사이로 부여하세요. 단, 특수공격의 성능이 좋다면 , 기본 능력치의 범위를 벗어날 만큼 약해도 괜찮습니다.
 특수공격 스킬은 캐릭터에 맞게 1개 부여하세요. 특수공격은 반드시 능력치에 관한 것이어야 합니다.또한 스킬은 반드시 전투에 영향을 미치는 것이어야 합니다.
 
 또한 턴 개념을 절대 언급하지 말 것 , 그리고 정의된 능력치 이외의 개념은 절대 언급하지 마세요.
 ###잘못된 예시 : '그림자 일섬: 2턴 동안 회피율을 65%로 고정하고, 지속 종료 시 단일 대상에게 공격력의 100%에 해당하는 관통 피해를 입힌다.'###
 ###잘못된 예시2 : '섀도우 스텝: 2턴 동안 자신의 회피율을 65%로 상승시키고 은신 상태에 돌입한다. 은신 중 첫 공격은 적의 방어력을 무시하며 공격력의 100%만큼 추가 피해를 입힌다.###
잘못된 부분 : 2턴, 지속, 단일 대상, 관통 피해, 은신 ,첫 공격, 추가 피해 등등 정의되지 않은 능력치와 턴 개념을 언급함

 올바른 예시 : '그림자 일섬: 자신의 회피율을 50% 증가시키고, 상대에게 자신의 공격력의 150%만큼 피해를 입힌다.'
올바른 이유: 턴 개념이 없고, 정의된 능력치인 회피율과 공격력에 관한 내용이기 때문입니다.
 """

examples = [
        {
        "input": "이 캐릭터는 판다야, 겉모습은 매우 귀엽지만 쿵푸를 매우 잘하고 어마무시한 방어력을 가지고 있지",
        "name" : "쿵푸판다",
        "hp": 250,
        "공격력": 20,
        "방어력": 20,
        "회피율": 10,
        "스킬": "본인의 몸을 단단하게 만들어 자신의 방어력을 50 증가시킨 후, 상대에게 해당 방어력의 100%만큼 피해를 입힌다.",
    },
    {
        "input": "이 캐릭터는 광전사야, 광전사는 매우 강력한 공격력을 가지고 있어",
        "name" : "버서커",
        "hp": 90,
        "공격력": 40,
        "방어력": 10,
        "회피율": 30,
        "스킬": "피가 60 이상으로 떨어지면 회피율이 50% 증가한다."
    },
    {
        "input": "이 캐릭터는 괴수야, 괴수는 매우 강한 체력과 방어력을 가지고 있어",
        "name" : "타이탄",
        "hp": 130,
        "공격력": 10,
        "방어력": 30,
        "회피율": 10,
        "스킬": "자신의 HP를 현재 HP의 30%만큼 회복하고, 상대에게 회복한 피해의 100%만큼 피해를 입힌다."
    }
]
json_parser = JsonOutputParser()
format_instructions = json_parser.get_format_instructions()

example_prompt = PromptTemplate(
    template = "input : {input}\nname : {name}, hp : {hp}, 공격력 : {공격력}, 방어력 : {방어력}, 스킬 : {스킬}, 회피율 : {회피율} \n",
    input_variables = ['input']
)

#json 형식지정
json_parser = JsonOutputParser()

fewshot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "당신은 캐릭터 생성기입니다. 사용자가 입력한 프롬프트에 맞는 캐릭터를 생성하고, 캐릭터에 맞는 능력치를 부여하세요. 능력치는 각각 hp, 공격력, 방어력,회피율과 특수공격을 위한 스킬이 주어집니다. 특수공격을 제외한 능력치는 다음과 같이 부여하세요. HP는 기본적으로 100~300이어야 하며 공격력은 20~40, 방어력은 10~30, 회피율은 10~30 사이로 부여하세요. 단, 특수공격의 성능이 좋다면 , 기본 능력치의 범위를 벗어날 만큼 약해도 괜찮습니다.특수공격 스킬은 캐릭터에 맞게 1개 부여하세요. 특수공격은 반드시 능력치에 관한 것이어야 합니다.또한 스킬은 반드시 전투에 영향을 미치는 것이어야 합니다.\n 형식지정 : {format_instructions}\n",
    suffix = "이제 새로운 캐릭터를 생성해봅시다. ###input : {input}###에 맞는 캐릭터를 생성하세요.",
    input_variables = ["input"],
    partial_variables={'format_instructions': format_instructions},
)
print(fewshot_prompt.format(input="이 캐릭터는 마법사야, 마법사는 강력한 공격력을 가지고 있어"))
# print(fewshot_prompt.format(input="이 캐릭터는 암살자야, 암살자는 특이한 스킬을 가지고 있어"))

당신은 캐릭터 생성기입니다. 사용자가 입력한 프롬프트에 맞는 캐릭터를 생성하고, 캐릭터에 맞는 능력치를 부여하세요. 능력치는 각각 hp, 공격력, 방어력,회피율과 특수공격을 위한 스킬이 주어집니다. 특수공격을 제외한 능력치는 다음과 같이 부여하세요. HP는 기본적으로 100~300이어야 하며 공격력은 20~40, 방어력은 10~30, 회피율은 10~30 사이로 부여하세요. 단, 특수공격의 성능이 좋다면 , 기본 능력치의 범위를 벗어날 만큼 약해도 괜찮습니다.특수공격 스킬은 캐릭터에 맞게 1개 부여하세요. 특수공격은 반드시 능력치에 관한 것이어야 합니다.또한 스킬은 반드시 전투에 영향을 미치는 것이어야 합니다.
 형식지정 : Return a JSON object.


input : 이 캐릭터는 판다야, 겉모습은 매우 귀엽지만 쿵푸를 매우 잘하고 어마무시한 방어력을 가지고 있지
name : 쿵푸판다, hp : 250, 공격력 : 20, 방어력 : 20, 스킬 : 본인의 몸을 단단하게 만들어 자신의 방어력을 50 증가시킨 후, 상대에게 해당 방어력의 100%만큼 피해를 입힌다., 회피율 : 10 


input : 이 캐릭터는 광전사야, 광전사는 매우 강력한 공격력을 가지고 있어
name : 버서커, hp : 90, 공격력 : 40, 방어력 : 10, 스킬 : 피가 60 이상으로 떨어지면 회피율이 50% 증가한다., 회피율 : 30 


input : 이 캐릭터는 괴수야, 괴수는 매우 강한 체력과 방어력을 가지고 있어
name : 타이탄, hp : 130, 공격력 : 10, 방어력 : 30, 스킬 : 자신의 HP를 현재 HP의 30%만큼 회복하고, 상대에게 회복한 피해의 100%만큼 피해를 입힌다., 회피율 : 10 


이제 새로운 캐릭터를 생성해봅시다. ###input : 이 캐릭터는 마법사야, 마법사는 강력한 공격력을 가지고 있어###에 맞는 캐릭터를 생성하세요.


In [3]:
llm = ChatOpenAI(model_name='gpt-5')
player1_prompt = input("플레이어1 캐릭터를 입력하세요 : ")

prompt = fewshot_prompt.format(input=player1_prompt)
ai_message = llm.invoke(prompt)

In [4]:
result = json_parser.parse(ai_message.content)
print(result)

{'name': '동현피티', 'hp': 210, '공격력': 28, '방어력': 22, '회피율': 24, '스킬': '약점 프로파일링: 대상의 공격력·방어력·회피율 중 가장 낮은 능력치를 2턴 동안 30% 감소시키고, 감소한 수치의 100%를 동일한 자신의 능력치에 2턴 동안 추가한다. 이후 감소한 수치의 100%만큼 대상에게 고정 피해를 입힌다. (재사용 대기시간: 3턴)'}


In [5]:
player2_prompt = input("플레이어2 캐릭터를 입력하세요 : ")
prompt = fewshot_prompt.format(input=player2_prompt)
ai_message = llm.invoke(prompt)

In [6]:
result2 = json_parser.parse(ai_message.content)
print(result)

{'name': '동현피티', 'hp': 210, '공격력': 28, '방어력': 22, '회피율': 24, '스킬': '약점 프로파일링: 대상의 공격력·방어력·회피율 중 가장 낮은 능력치를 2턴 동안 30% 감소시키고, 감소한 수치의 100%를 동일한 자신의 능력치에 2턴 동안 추가한다. 이후 감소한 수치의 100%만큼 대상에게 고정 피해를 입힌다. (재사용 대기시간: 3턴)'}


In [7]:
class Player:
    def __init__(self, name, hp, attack, defense, skill,evade):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.skill = skill
        self.evade = evade

    def __str__(self):
        return f"캐릭터 이름: {self.name}\n체력: {self.hp}\n공격력: {self.attack}\n방어력: {self.defense}\n회피율: {self.evade}\n특수공격 스킬: {self.skill}"

# 여기서 두 명이서 각자 원하는 캐릭터 입력하시면 됩니다.

- ex) 세상에서 가장 귀여운 고양이 키티, 이 고양이는 애교로 세상을 멸망시킬 존재이다.

- ex) 악어와 기관총을 합친 악어들의 왕, 봄바르딜로 크로코딜로 그의 기관총은 모든 적을 무자비하게 죽여버린다.


### 캐릭터 1

In [8]:
player1 = Player(result['name'], result['hp'], result['공격력'], result['방어력'], result['스킬'],result['회피율'])
print(player1)

캐릭터 이름: 동현피티
체력: 210
공격력: 28
방어력: 22
회피율: 24
특수공격 스킬: 약점 프로파일링: 대상의 공격력·방어력·회피율 중 가장 낮은 능력치를 2턴 동안 30% 감소시키고, 감소한 수치의 100%를 동일한 자신의 능력치에 2턴 동안 추가한다. 이후 감소한 수치의 100%만큼 대상에게 고정 피해를 입힌다. (재사용 대기시간: 3턴)


#### 캐릭터 2

In [9]:
player2 = Player(result2['name'], result2['hp'], result2['공격력'], result2['방어력'], result2['스킬'],result2['회피율'])
print(player2)

캐릭터 이름: 평범한 남자 1
체력: 180
공격력: 26
방어력: 14
회피율: 14
특수공격 스킬: 평범의 역전: 전투 중 1회 사용. 상대의 공격력/방어력/회피율 중 현재 가장 높은 능력치를 찾아 2턴 동안 그 능력치를 서로 교환한다(동률일 경우 공격력>방어력>회피율 우선). 교환 직후, 교환된 해당 능력치의 절대 차이만큼 추가 고정 피해를 즉시 입힌다. 2턴이 지나면 양쪽 능력치는 원래 값으로 복귀한다. 재사용 불가.


### 특수공격 스킬 -> def

In [10]:
# ...existing code...
import json
from typing import Tuple, List

def skill(attacker: Player, defender: Player, llm) -> str:
    # 프롬프트: 이번 턴 변화량(delta)만 요구
    format_instructions = """
반드시 아래 JSON 하나만 출력하세요:
{
  "attacker_delta": {"hp": int, "attack": int, "defense": int, "evade": int},
  "defender_delta": {"hp": int, "attack": int, "defense": int, "evade": int},
  "log": [string, ...]
}
규칙:
- delta는 변화량(가감)입니다. 예: hp:-20은 피해 20, hp:+15는 회복 15.
- 숫자는 정수만. 없는 항목은 0으로.
- 반사는 defender 스킬에 '반사'/'되돌리'가 있을 때 10~50% 내에서 합리적으로.
- 근거 없는 과도한 버프/디버프는 피하세요.
"""

    prompt = f"""
당신은 턴제 RPG의 특수스킬 해석기입니다.
두 캐릭터의 현재 능력치와 스킬 설명을 보고, 이번 턴에만 적용될 변화량(delta)을 산출해 주세요.

attacker(공격자) 현재 상태:
{{
  "name": "{attacker.name}",
  "hp": {attacker.hp}, "attack": {attacker.attack}, "defense": {attacker.defense}, "evade": {attacker.evade},
  "skill": "{attacker.skill}"
}}

defender(방어자) 현재 상태:
{{
  "name": "{defender.name}",
  "hp": {defender.hp}, "attack": {defender.attack}, "defense": {defender.defense}, "evade": {defender.evade},
  "skill": "{defender.skill}"
}}

해석 지침:
- 스킬 문구에 근거해서 회복/버프/디버프/반사/회피 등을 판단하세요.
- 수치가 명시되지 않으면 상식적인 범위로 보수적으로 가정하세요.
- 출력은 delta만 반환하고, 최종 수치는 코드가 적용합니다.
- attacker의 스킬 적용 후 defender의 스킬이 발동할 수 있습니다. 양쪽 스킬을 모두 고려하세요.
- 만약 조건을 충족해야만 발동하는 스킬이라면, 현재 상태로 발동하지 않는다면 모두 0으로 출력하세요.

예시:
input:
attacker(공격자) 현재 상태:
        "name" : "쿵푸판다",
        "hp": 90,
        "공격력": 50,
        "방어력": 90,
        "회피율": 70,
        "스킬": "본인의 몸을 단단하게 만들어 자신의 방어력을 50 증가시킨 후, 상대에게 해당 방어력의 10%만큼 피해를 입힌다."

defender(방어자) 현재 상태:
        "name": "버서커",
        "hp": 20,
        "공격력": 180,
        "방어력": 10,
        "회피율": 90,
        "스킬": "피가 5 이상으로 떨어지면 회피율이 50% 증가한다."

output:
attacker_delta: {{"hp": 0, "attack": 0, "defense": 50, "evade": 0}}
defender_delta: {{"hp": -5, "attack": 0, "defense": 0, "evade": 50}}

예시2:
input:
attacker(공격자) 현재 상태:
        "name": "버서커",
        "hp": 20,
        "공격력": 180,
        "방어력": 10,
        "회피율": 90,
        "스킬": "피가 5 이상으로 떨어지면 회피율이 50% 증가한다."

defender(방어자) 현재 상태:
        "name" : "쿵푸판다",
        "hp": 90,
        "공격력": 50,
        "방어력": 90,
        "회피율": 70,
        "스킬": "본인의 몸을 단단하게 만들어 자신의 방어력을 50 증가시킨 후, 상대에게 해당 방어력의 100%만큼 피해를 입힌다."

output:
attacker_delta: {{"hp": 0, "attack": 0, "defense": 0, "evade": 0}}
defender_delta: {{"hp": 0, "attack": 0, "defense": 0, "evade": 0}}


{format_instructions}
""".strip()

    # LLM 호출
    ai = llm.invoke(prompt)
    try:
        data = json.loads(ai.content)
    except Exception:
        # JSON이 아닐 경우 안전 기본값
        data = {
            "attacker_delta": {"hp": 0, "attack": 0, "defense": 0, "evade": 0},
            "defender_delta": {"hp": 0, "attack": 0, "defense": 0, "evade": 0},
            "log": ["스킬 해석 실패: 기본값 적용"]
        }

    def get_delta(side: str, key: str) -> int:
        try:
            return int(data.get(f"{side}_delta", {}).get(key, 0))
        except Exception:
            return 0

    # delta 추출 및 클램프
    a_hp =  get_delta("attacker", "hp")
    d_hp =  get_delta("defender", "hp")
    a_atk = get_delta("attacker", "attack")
    a_def = get_delta("attacker", "defense")
    a_eva = get_delta("attacker", "evade")
    d_atk = get_delta("defender", "attack")
    d_def = get_delta("defender", "defense")
    d_eva = get_delta("defender", "evade")

    # 적용
    attacker.hp = max(0, attacker.hp + a_hp)
    defender.hp = max(0, defender.hp + d_hp)

    attacker.attack = max(0, attacker.attack + a_atk)
    attacker.defense = max(0, attacker.defense + a_def)
    attacker.evade = int(max(0, min(100, attacker.evade + a_eva)))

    defender.attack = max(0, defender.attack + d_atk)
    defender.defense = max(0, defender.defense + d_def)
    defender.evade = int(max(0, min(100, defender.evade + d_eva)))

    # 로그
    logs: List[str] = data.get("log", [])
    if not isinstance(logs, list):
        logs = [str(logs)]
    # 적용 결과 요약 추가
    logs.append(
        f"적용 결과 → {attacker.name} HP:{attacker.hp}, ATK:{attacker.attack}, DEF:{attacker.defense}, EVA:{attacker.evade} | "
        f"{defender.name} HP:{defender.hp}, ATK:{defender.attack}, DEF:{defender.defense}, EVA:{defender.evade}"
    )
    return "\n".join(logs)



#### Player 1 VS Player 2

In [17]:
# ...existing code...
import random
from copy import deepcopy

class Battle:
    def __init__(self,p1: Player,p2: Player,llm=None,seed: int | None = None):
        self.p1 = deepcopy(p1)
        self.p2 = deepcopy(p2)
        self.turn = 1
        self.llm = llm
        # 각 캐릭터의 마지막 스킬 사용 턴(초기에는 매우 과거로 설정하여 1턴에 사용 가능)
        self.remained_skill_count = {"p1": 1, "p2": 1}
        self.cumulative_damage = {"p1": 0, "p2": 0}
        # ...existing code...
        if seed is not None:
            random.seed(seed)

    #데미지 공식
    def calc_damage(self, attacker: Player, defender: Player) -> int:
        base = max(1, int(attacker.attack - defender.defense / 3))
        luck = random.randint(0, 5)
        return max(1, base + luck)

    #회피 판정
    def check_evasion(self, defender: Player) -> tuple[bool, str]:
        """defender의 회피율(%)을 바탕으로 회피 여부 판정"""
        raw_ev = getattr(defender, 'evade', 0)
        try:
            raw_ev = float(raw_ev)
        except Exception:
            raw_ev = 0.0
        chance = max(0.0, min(0.95, raw_ev / 100.0))  # 0~95% 클램프

        if random.random() < chance:
            percent = int(round(chance * 100))
            return True, f"{defender.name}이(가) 회피율 {percent}%로 회피!"
        return False, ""
    
    def is_over(self) -> bool:
        return self.p1.hp <= 0 or self.p2.hp <= 0

    def winner(self) -> Player | None:
        if self.p1.hp <= 0 and self.p2.hp <= 0:
            return None
        if self.p1.hp <= 0:
            return self.p2
        if self.p2.hp <= 0:
            return self.p1
        return None
    
    def simulate(self):
        while not self.is_over():
            print(f"\n--- {self.turn}턴 시작 ---")
            for attacker, defender, key in [(self.p1, self.p2, "p1"), (self.p2, self.p1, "p2")]:
                if self.is_over():
                    break

                print(f"\n{attacker.name}의 턴:")
                print()

                 #현재 상태
                print("현재 상태:")
                print(f"{attacker.name} HP: {attacker.hp}, ATK: {attacker.attack}, DEF: {attacker.defense}, EVA: {attacker.evade}")
                print(f"{defender.name} HP: {defender.hp}, ATK: {defender.attack}, DEF: {defender.defense}, EVA: {defender.evade}")
                print()
                
                #누적데미지
                can_use_skill = (self.remained_skill_count[key] >= 1 and self.cumulative_damage[key] >= 70)
                if not can_use_skill:
                    print(f"{attacker.name}의 누적 피해: {self.cumulative_damage[key]} (스킬 조건: 누적 70 이상)")
                    print()
                    is_using_skill = 'n'
                    out_point = input(f"{attacker.name}님 일반 공격을 사용하시겠습니까? (y): ")
                    if out_point.lower() == 'q':
                        print("전투를 종료합니다.")
                        return
                else:
                    is_using_skill = input(f"{attacker.name}님 스킬을 사용하시겠습니까? (y/n): ")
                
                # 스킬 사용 가능 여부 판단
                if is_using_skill.lower() == 'y' and can_use_skill:
                    print(f"{attacker.name}이(가) 스킬을 사용합니다: {attacker.skill}")
                    log = skill(attacker, defender, self.llm)
                    print(log)
                    # 한 번만 사용 가능
                    self.remained_skill_count[key] -= 1
                else:
                    if is_using_skill.lower() == 'y' and not can_use_skill:
                        reason = "이미 스킬을 사용했습니다." if self.remained_skill_count[key] < 1 else "누적 피해가 70 미만입니다."
                        print(f"{attacker.name}은(는) {reason} 일반 공격으로 전환합니다.")
                    damage = self.calc_damage(attacker, defender)
                    evaded, evasion_log = self.check_evasion(defender)
                    if evaded:
                        print(evasion_log)
                        print(f"{defender.name}이(가) 공격을 회피했습니다!")
                    else:
                        defender.hp = max(0, defender.hp - damage)
                        # 직전 피해 및 누적 피해 업데이트
                        self.cumulative_damage[key] += damage
                        print(f"{attacker.name}이(가) {defender.name}에게 {damage}의 피해를 입혔습니다! ({defender.name} HP: {defender.hp})")
                        print(f"누적 피해 업데이트: {attacker.name} = {self.cumulative_damage[key]}")


            self.turn += 1

        print("\n--- 전투 종료 ---")
        w = self.winner()
        if w is None:
            print("무승부!")
        else:
            print(f"승자: {w.name}")
    

# 게임 실행

In [18]:
battle = Battle(player1, player2, llm=llm, seed=42)
battle.simulate()


--- 1턴 시작 ---

동현피티의 턴:

현재 상태:
동현피티 HP: 210, ATK: 28, DEF: 22, EVA: 24
평범한 남자 1 HP: 180, ATK: 26, DEF: 14, EVA: 14

동현피티의 누적 피해: 0 (스킬 조건: 누적 70 이상)

평범한 남자 1이(가) 회피율 14%로 회피!
평범한 남자 1이(가) 공격을 회피했습니다!

평범한 남자 1의 턴:

현재 상태:
평범한 남자 1 HP: 180, ATK: 26, DEF: 14, EVA: 14
동현피티 HP: 210, ATK: 28, DEF: 22, EVA: 24

평범한 남자 1의 누적 피해: 0 (스킬 조건: 누적 70 이상)

평범한 남자 1이(가) 동현피티에게 23의 피해를 입혔습니다! (동현피티 HP: 187)
누적 피해 업데이트: 평범한 남자 1 = 23

--- 2턴 시작 ---

동현피티의 턴:

현재 상태:
동현피티 HP: 187, ATK: 28, DEF: 22, EVA: 24
평범한 남자 1 HP: 180, ATK: 26, DEF: 14, EVA: 14

동현피티의 누적 피해: 0 (스킬 조건: 누적 70 이상)

평범한 남자 1이(가) 회피율 14%로 회피!
평범한 남자 1이(가) 공격을 회피했습니다!

평범한 남자 1의 턴:

현재 상태:
평범한 남자 1 HP: 180, ATK: 26, DEF: 14, EVA: 14
동현피티 HP: 187, ATK: 28, DEF: 22, EVA: 24

평범한 남자 1의 누적 피해: 23 (스킬 조건: 누적 70 이상)

평범한 남자 1이(가) 동현피티에게 18의 피해를 입혔습니다! (동현피티 HP: 169)
누적 피해 업데이트: 평범한 남자 1 = 41

--- 3턴 시작 ---

동현피티의 턴:

현재 상태:
동현피티 HP: 169, ATK: 28, DEF: 22, EVA: 24
평범한 남자 1 HP: 180, ATK: 26, DEF: 14, EVA: 14

동현피티의 누적 피해: 0 (스킬 조건: 누적 70 이상)


## 이기신 분의 프롬프트는 그대로 남기고 진 분만 다른 걸로 바꾸시면 됩니다.